In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV, cross_val_score, RandomizedSearchCV
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.decomposition import PCA
from torch.utils.data import TensorDataset, DataLoader
from scipy.stats import uniform, randint, loguniform

In [2]:
data = pd.read_csv("./train.csv")
X = data.iloc[:, 6:]
classifiers = data.iloc[:, 1:4]
enc = OneHotEncoder()
data_oh = enc.fit_transform(classifiers).toarray()
data_oh = pd.DataFrame(data_oh)


In [7]:
#spectrum_filtered = pd.DataFrame(savgol_filter(spectrum, 7, 3, deriv = 2, axis = 0))
#spectrum_filtered_st = zscore(spectrum_filtered, axis = 1)

y = data["PURITY"]/100
print(y)

pca = PCA(n_components=16)
X_pca = pd.DataFrame(pca.fit_transform(X))
print(pca.explained_variance_ratio_) #pour trouver le nombre de composantes à garder
X_new = pd.concat([data_oh, X_pca], axis=1)
X_new.columns = X_new.columns.astype(str)

X_train, X_valid, y_train, y_valid = train_test_split(X_new, y, test_size=0.2, random_state=42)

standardizer = StandardScaler()
X_train_standardized = standardizer.fit_transform(X_train)
X_valid_standardized = standardizer.transform(X_valid)

# Convert to tensors
X_train_tensor = torch.tensor(X_train_standardized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
X_valid_tensor = torch.tensor(X_valid_standardized, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float32).reshape(-1, 1)

0       0.585000
1       0.186000
2       0.198817
3       0.525000
4       0.489000
          ...   
1295    0.101000
1296    0.449000
1297    0.534000
1298    0.136000
1299    0.196000
Name: PURITY, Length: 1300, dtype: float64
[8.63385983e-01 1.19426741e-01 1.42997513e-02 1.98910919e-03
 4.96149909e-04 1.60475196e-04 9.86566323e-05 4.91435637e-05
 3.35702776e-05 1.86516555e-05 1.37499171e-05 8.14845972e-06
 4.42312642e-06 3.64445868e-06 2.58705888e-06 2.06030889e-06]


In [9]:
# Définir le modèle de réseau de neurones simple
class SimpleNN(nn.Module):
    def __init__(self, input_size, fc1_out_features=50, fc2_out_features=1, dropout_rate=0.2):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, fc1_out_features)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(fc1_out_features, fc2_out_features)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(fc2_out_features, 1)
    
    def forward(self, x):
        x = F.silu(self.fc1(x))
        x = self.dropout1(x)
        x = F.silu(self.fc2(x))
        x = self.dropout2(x)
        x = F.sigmoid(self.fc3(x))
        return x

# Définir la classe NeuralNetRegressor
class NeuralNetRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, input_size, random_state, eta=0.001, max_epochs=100, fc1_out_features=50, fc2_out_features=1, batch=10, dropout_rate = 0.5):
        self.input_size = input_size
        self.random_state = random_state
        self.eta = eta
        self.max_epochs = max_epochs
        self.fc1_out_features = fc1_out_features
        self.fc2_out_features = fc2_out_features
        self.batch = batch
        self.dropout_rate = dropout_rate
        self.model = SimpleNN(input_size, fc1_out_features, fc2_out_features)
        self.criterion = nn.MSELoss()
    
    def fit(self, X, y, do_print=False):
        optimizer = optim.Adam(self.model.parameters(), lr=self.eta)
        X_tensor = torch.tensor(X).clone().detach().float()
        y_tensor = torch.tensor(y).clone().detach().float()
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=self.batch, shuffle=True)
        self.model.train()
        # Training loop
        for epoch in range(self.max_epochs):
            epoch_loss = 0.0
            for batch_X, batch_y in dataloader:
                optimizer.zero_grad()  # Reset gradients
                outputs = self.model(batch_X)  # Forward pass
                loss = self.criterion(outputs, batch_y)  # Compute loss
                loss.backward()  # Backward pass
                optimizer.step()  # Update parameters
                epoch_loss += loss.item()
            if do_print:
                print(f"Epoch {epoch+1}/{self.max_epochs}, Loss: {epoch_loss / len(dataloader)}")
        return self
    
    def predict(self, X):
        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32)
        with torch.no_grad():
            outputs = self.model(X_tensor).flatten()
        return outputs.numpy()
    
    def parameters(self):
        return self.model.parameters()

In [20]:
# Initialiser le modèle
input_size = X_train_tensor.shape[1]
net = NeuralNetRegressor(input_size=input_size)

# Définir les paramètres pour GridSearch
params = {
    'eta': [0.0005, 0.001],
    'max_epochs': [50],
    'fc1_out_features': [32, 64, 128],
    'fc2_out_features': [32, 64, 128],
    'batch': [65, 70],
    'dropout_rate': [0.3, 0.4, 0.5]
}

# Initialiser GridSearchCV
grid_search = GridSearchCV(net, params, refit=True, cv=5, scoring='neg_mean_squared_error', verbose=0)

# Entraîner le modèle avec GridSearch
grid_result = grid_search.fit(X_train_tensor, y_train_tensor)
nouveau_model = grid_result.best_estimator_

print("Best MSE: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_1170/3745802480.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X).clone().detach().float()
/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_1170/3745802480.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_tensor = torch.tensor(y).clone().detach().float()
/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_1170/3745802480.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X, dtype=torch.float32)
/var/fol

KeyboardInterrupt: 

In [10]:
# Initialiser le modèle
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

input_size = X_train_tensor.shape[1]
net = NeuralNetRegressor(input_size=input_size, random_state=42)

# Définir les paramètres pour GridSearch
params_dist = {
    'eta': loguniform(1e-4, 1e-1),
    'max_epochs': randint(50, 150),
    'fc1_out_features': randint(32, 128),
    'fc2_out_features': randint(32, 128),
    'batch': randint(32, 70),
    'dropout_rate': uniform(0,0.5)
}

# Initialiser RandomizedSearchCV
random_search = RandomizedSearchCV(net, params_dist, refit=True, cv=5, random_state=42, scoring='neg_mean_squared_error', verbose=0)

# Entraîner le modèle avec GridSearch
random_grid_result = random_search.fit(X_train_tensor, y_train_tensor)
nouveau_model = random_grid_result.best_estimator_

print("Best MSE: %f using %s" % (random_grid_result.best_score_, random_grid_result.best_params_))

/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_956/2707888768.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X).clone().detach().float()
/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_956/2707888768.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_tensor = torch.tensor(y).clone().detach().float()
/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_956/2707888768.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X, dtype=torch.float32)
/var/folder

Best MSE: -0.003769 using {'batch': 35, 'dropout_rate': 0.4711008778424264, 'eta': 0.004896262051737688, 'fc1_out_features': 40, 'fc2_out_features': 121, 'max_epochs': 102}


In [11]:
y_pred = nouveau_model.predict(X_valid_tensor)
y_pred_train=nouveau_model.predict(X_train_tensor)

# Vérifiez les sorties du modèle
print("y_pred contains NaN:", np.isnan(y_pred).any())

# Calculer la MSE
mse = np.mean(((y_pred - y_valid)*100)**2)
print("MSE :", mse)

# Calculer le t_score
train_score = np.mean(np.abs((y_pred_train-y_train)*100<=5))
test_score = np.mean(np.abs((y_pred-y_valid)*100<=5))
print("t_score test :", train_score)
print("t_score train :", test_score)

"""
# Afficher la courbe d'apprentissage
import matplotlib.pyplot as plt
plt.plot(steps, learning_curve)
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.show()
"""

y_pred contains NaN: False
MSE : 40.38185119297534
t_score test : 0.9538461538461539
t_score train : 0.9307692307692308


/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_956/2707888768.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X, dtype=torch.float32)


"\n# Afficher la courbe d'apprentissage\nimport matplotlib.pyplot as plt\nplt.plot(steps, learning_curve)\nplt.xlabel('Steps')\nplt.ylabel('Loss')\nplt.title('Learning Curve')\nplt.show()\n"